# Traitement des données

In [5]:
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
import numpy as np
import json
from geopy.distance import geodesic
from pathlib import Path
import warnings
import os

In [6]:
files_to_check = [
    'data/philly_restaurants.json',
    'data/philly_restaurant_reviews.json',
    'data/philly_restaurant_users.json',
    'data/philly_restaurant_tips.json',
    'data/philly_restaurant_photos.json'
]


def load_json_data(filepath):
    """Charge un fichier JSON ligne par ligne (format Yelp)"""
    data = []
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line))
        print(f"{filepath}: {len(data)} enregistrements chargés")
        return pd.DataFrame(data)
    except FileNotFoundError:
        print(f"{filepath}: Fichier non trouvé")
        return None
    except Exception as e:
        print(f"{filepath}: Erreur - {e}")
        return None

# Chargement des datasets
business_df = load_json_data(files_to_check[0])
reviews_df = load_json_data(files_to_check[1])
users_df = load_json_data(files_to_check[2])
tips_df = load_json_data(files_to_check[3])
photos_df = load_json_data(files_to_check[4])

data/philly_restaurants.json: 5852 enregistrements chargés
data/philly_restaurant_reviews.json: 687289 enregistrements chargés
data/philly_restaurant_users.json: 209513 enregistrements chargés
data/philly_restaurant_tips.json: 87118 enregistrements chargés
data/philly_restaurant_photos.json: 22295 enregistrements chargés


## Vision globale de la bdd

In [7]:
def analyze_dataframe(df, name):
    """Analyse descriptive complète"""
    if df is None:
        return

    print(f"\n{'─' * 80}")
    print(f" TABLE: {name}")
    print(f"{'─' * 80}")

    # Dimensions
    print(f"\n Dimensions: {df.shape[0]:,} lignes × {df.shape[1]} colonnes")

    # Colonnes et types
    print(f"\n Colonnes et types de données:")
    print(df.dtypes)

    # Aperçu des premières lignes
    print(f"\n Aperçu des données:")
    print(df.head(3))

    # Statistiques descriptives (colonnes numériques)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\n Statistiques descriptives (colonnes numériques):")
        print(df[numeric_cols].describe())

    # Valeurs manquantes
    print(f"\n Valeurs manquantes:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Valeurs manquantes': missing,
        'Pourcentage (%)': missing_pct
    })
    missing_df = missing_df[missing_df['Valeurs manquantes'] > 0].sort_values(
        'Valeurs manquantes', ascending=False
    )
    if len(missing_df) > 0:
        print(missing_df)
    else:
        print("Aucune valeur manquante ")

    # Mémoire utilisée
    memory_usage = df.memory_usage(deep=True).sum() / 1024**2
    print(f"\n Mémoire utilisée: {memory_usage:.2f} MB")

# Analyse de chaque table
if business_df is not None:
    analyze_dataframe(business_df, "BUSINESS")
if reviews_df is not None:
    analyze_dataframe(reviews_df, "REVIEWS")
if users_df is not None:
    analyze_dataframe(users_df, "USERS")
if tips_df is not None:
    analyze_dataframe(tips_df, "TIPS")
if photos_df is not None:
    analyze_dataframe(photos_df, "PHOTOS")



────────────────────────────────────────────────────────────────────────────────
 TABLE: BUSINESS
────────────────────────────────────────────────────────────────────────────────

 Dimensions: 5,852 lignes × 14 colonnes

 Colonnes et types de données:
business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
dtype: object

 Aperçu des données:
              business_id                name        address          city  \
0  MTSW4McQd7CbVtyjqoe9mw  St Honore Pastries    935 Race St  Philadelphia   
1  MUTTqe8uqyMdBl186RmNeA            Tuna Bar    205 Race St  Philadelphia   
2  ROeacJQwBeh05Rqg7F6TCg                 BAP  1224 South St  Philadelphia   

  state postal_code   latitude  longitude  stars  review_count

Pour la géographie :
Tu es bien géographiquement cohérent (Philadelphie)
Pas d’outliers absurdes
La dispersion est faible -> OK pour analyses spatiales

Pour les notes :
La médiane est plus haute que la moyenne → asymétrie à gauche
Beaucoup de restaurants sont notés 4+
Les très mauvaises notes tirent la moyenne vers le bas

Pour le nombre de reviews :
50 % des restaurants ont ≤ 19 reviews
75 % ont ≤ 53 reviews
mais certains ont des milliers
Beaucoup de restos peu visibles et quelques restos ultra-exposés (ça va nous etre utile du coup)

In [8]:
overview_table = pd.DataFrame({
    'dataset': ['business_df', 'reviews_df', 'users_df', 'tips_df', 'photos_df'],
    'n_rows': [
        len(business_df),
        len(reviews_df),
        len(users_df),
        len(tips_df),
        len(photos_df)
    ],
    'n_columns': [
        business_df.shape[1],
        reviews_df.shape[1],
        users_df.shape[1],
        tips_df.shape[1],
        photos_df.shape[1]
    ]
})

overview_table


,dataset,n_rows,n_columns
0,business_df,5852,14
1,reviews_df,687289,9
2,users_df,209513,22
3,tips_df,87118,5
4,photos_df,22295,4


## Filtrage des restaurants

In [6]:
if business_df is not None:
    print(f"\n Nombre total de commerces: {len(business_df):,}")

    # Afficher les catégories disponibles
    if 'categories' in business_df.columns:
        print("\n Exemples de catégories:")
        print(business_df['categories'].dropna().head(10))

    # Filtrer les restaurants
    # Méthode 1: Via la colonne 'categories'
    if 'categories' in business_df.columns:
        restaurants_df = business_df[
            business_df['categories'].notna() &
            business_df['categories'].str.contains('Restaurant', case=False, na=False)
        ].copy()

        print(f"\n Nombre de restaurants identifiés: {len(restaurants_df):,}")
        print(f"   Pourcentage: {len(restaurants_df)/len(business_df)*100:.1f}%")

        # Sous-catégories de restaurants les plus fréquentes
        if len(restaurants_df) > 0:
            print("\n Top 15 catégories de restaurants:")
            all_categories = restaurants_df['categories'].str.split(', ', expand=True).stack()
            top_categories = all_categories.value_counts().head(15)
            for cat, count in top_categories.items():
                print(f"   {cat}: {count:,}")
    else:
        print(" Colonne 'categories' non trouvée")
        restaurants_df = business_df.copy()


 Nombre total de commerces: 14,569

 Exemples de catégories:
0    Restaurants, Food, Bubble Tea, Coffee & Tea, B...
1                    Sushi Bars, Restaurants, Japanese
2                                  Korean, Restaurants
3    Cocktail Bars, Bars, Italian, Nightlife, Resta...
4                      Pizza, Restaurants, Salad, Soup
5    Eatertainment, Arts & Entertainment, Brewpubs,...
6                                        Food, Grocery
7    Restaurants, Automotive, Delis, Gas Stations, ...
8     Keys & Locksmiths, Home Services, Local Services
9                        Eyewear & Opticians, Shopping
Name: categories, dtype: object

 Nombre de restaurants identifiés: 5,854
   Pourcentage: 40.2%

 Top 15 catégories de restaurants:
   Restaurants: 5,852
   Food: 1,886
   Nightlife: 1,036
   Bars: 980
   Sandwiches: 929
   Pizza: 800
   American (New): 751
   Breakfast & Brunch: 686
   American (Traditional): 663
   Coffee & Tea: 575
   Italian: 505
   Chinese: 471
   Fast Food: 367
 

## Nettoyage des avis

In [11]:
if reviews_df is not None and business_df is not None:
    # Joindre les avis aux restaurants
    restaurant_reviews = reviews_df.merge(
        restaurants_df[['business_id', 'name', 'categories']],
        on='business_id',
        how='inner'
    )

    print(f"\n Avis de restaurants: {len(restaurant_reviews):,}")
    print(f"   (sur {len(reviews_df):,} avis totaux)")

    # Statistiques sur les avis
    print("\n Statistiques des avis:")
    print(f"   Note moyenne: {restaurant_reviews['stars'].mean():.2f}/5")
    print(f"   Médiane: {restaurant_reviews['stars'].median():.1f}/5")
    print(f"   Écart-type: {restaurant_reviews['stars'].std():.2f}")

    print("\n Distribution des notes:")
    stars_dist = restaurant_reviews['stars'].value_counts().sort_index()
    for star, count in stars_dist.items():
        pct = count / len(restaurant_reviews) * 100
        bar = '█' * int(pct / 2)
        print(f"   {star}: {count:>7,} ({pct:>5.1f}%) {bar}")

    # Longueur des avis
    if 'text' in restaurant_reviews.columns:
        restaurant_reviews['review_length'] = restaurant_reviews['text'].str.len()

        print("\n Longueur des avis:")
        print(f"   Moyenne: {restaurant_reviews['review_length'].mean():.0f} caractères")
        print(f"   Médiane: {restaurant_reviews['review_length'].median():.0f} caractères")
        print(f"   Min: {restaurant_reviews['review_length'].min():.0f}")
        print(f"   Max: {restaurant_reviews['review_length'].max():.0f}")

        # Nettoyer les avis
        print("\n Nettoyage des avis:")
        initial_count = len(restaurant_reviews)

        # Supprimer les avis vides ou trop courts
        restaurant_reviews = restaurant_reviews[
            (restaurant_reviews['text'].notna()) &
            (restaurant_reviews['review_length'] >= 10)
        ]
        print(f"   Avis supprimés (vides/trop courts): {initial_count - len(restaurant_reviews):,}")

        # Prétraitement basique du texte
        restaurant_reviews['text_clean'] = restaurant_reviews['text'].str.lower()
        restaurant_reviews['text_clean'] = restaurant_reviews['text_clean'].str.replace(
            r'[^\w\s]', ' ', regex=True
        )
        restaurant_reviews['text_clean'] = restaurant_reviews['text_clean'].str.replace(
            r'\s+', ' ', regex=True
        ).str.strip()


 Avis de restaurants: 687,307
   (sur 967,552 avis totaux)

 Statistiques des avis:
   Note moyenne: 3.81/5
   Médiane: 4.0/5
   Écart-type: 1.31

 Distribution des notes:
   1.0:  66,626 (  9.7%) ████
   2.0:  57,480 (  8.4%) ████
   3.0:  91,706 ( 13.3%) ██████
   4.0: 194,373 ( 28.3%) ██████████████
   5.0: 277,122 ( 40.3%) ████████████████████

 Longueur des avis:
   Moyenne: 602 caractères
   Médiane: 443 caractères
   Min: 1
   Max: 5000

 Nettoyage des avis:
   Avis supprimés (vides/trop courts): 18


# Features globales

In [13]:
restaurant_reviews = restaurant_reviews.copy()

restaurant_reviews['date'] = pd.to_datetime(
    restaurant_reviews['date'],
    errors='coerce'
)

# Vérification
print(restaurant_reviews['date'].dtype)

# Supprimer les dates invalides
restaurant_reviews = restaurant_reviews.dropna(subset=['date'])

datetime64[ns]


## Features de popularité

In [12]:
restaurant_reviews['date'].dtype

dtype('O')

In [14]:
restaurant_features = (
    restaurant_reviews
    .groupby('business_id')
    .agg(
        n_reviews=('stars', 'count'),
        avg_stars=('stars', 'mean'),
        std_stars=('stars', 'std'),
        first_review=('date', 'min'),
        last_review=('date', 'max')
    )
    .reset_index()
)

restaurant_features['std_stars'] = restaurant_features['std_stars'].fillna(0)

# calculer span en jours
restaurant_features['activity_span_days'] = (
    (restaurant_features['last_review'] - restaurant_features['first_review']).dt.days
).clip(lower=1)

# calculer velocity
restaurant_features['velocity_reviews'] = (
    restaurant_features['n_reviews'] / restaurant_features['activity_span_days']
)

- n_reviews : Nombre d’avis réellement utilisés après nettoyage. En gros c'est la quantité d’information fiable dont tu disposes.
Ce qui est cool du coup c'est que faible volume + bonnes notes ça pourrait dire une pépite potentielle et un gros volume ça veut dire un établissement très exposé (tourisme, chaînes, etc)
- avg_stars : moyenne (Globalement, c’est bien ou pas ?)
- std_stars : ecart type (est qu'ils sont d'accord)

- activity_span_days : ça mesure le nombre de jours entre le premier et le dernier avis.
- velocity_reviews : très élevé -> lieu récemment “découvert” ou sur-exposé. C'est typique des spots touristiques
Ca nous permet de mesurer la durabilité du restaurant et de différencier hype récente vs qualité stable

## Distribution des notes

In [15]:
stars_dist = (
    restaurant_reviews
    .groupby(['business_id', 'stars'])
    .size()
    .unstack(fill_value=0)
)

stars_dist.columns = [f'n_star_{int(c)}' for c in stars_dist.columns]
row_total = stars_dist.sum(axis=1, numeric_only=True)

stars_dist['share_5_star'] = stars_dist['n_star_5'] / row_total
stars_dist['share_1_star'] = stars_dist['n_star_1'] / row_total

- n_star_... : nb d'avis par niveau de note (c'est pour mieux détecter les extremes)

On se penche en particulier sur share_5_star et share_1_star pour pouvoir détecter l'insatifaction severe ou l'enthousiasme fort

## Features temporelles

In [16]:
restaurant_reviews['date'] = pd.to_datetime(restaurant_reviews['date'])

time_features = (
    restaurant_reviews
    .groupby('business_id')
    .agg(
        first_review=('date', 'min'),
        last_review=('date', 'max')
    )
    .reset_index()
)

time_features['activity_span_days'] = (
    (time_features['last_review'] - time_features['first_review']).dt.days
).clip(lower=1)

time_features['reviews_per_month'] = (
    restaurant_features['n_reviews'] /
    (time_features['activity_span_days'] / 30).clip(lower=1)
)

## Saisonalité

In [35]:
restaurant_reviews['month'] = restaurant_reviews['date'].dt.month

seasonality = (
    restaurant_reviews
    .groupby(['business_id', 'month'])
    .size()
    .groupby('business_id')
    .std()
    .reset_index(name='monthly_review_volatility')
)

Pour chaque restaurant, tu prends l’écart-type du nombre de reviews mensuelles.
- faible std -> le restaurant reçoit à peu près le même nombre de reviews chaque mois
- forte std -> forte variation selon le mois (saisonnalité marquée)

## Création d'un dataset avec juste les features

In [36]:
features_df = (
    restaurant_features
    .merge(stars_dist, on='business_id', how='left')
    .merge(seasonality, on='business_id', how='left')
    .merge(time_features, on='business_id', how='left')
    .merge(restaurants_df, on='business_id', how='left')
)


# Feature géographiques

## Densité locale pour la concu

In [40]:
geo_df = restaurants_df[['business_id', 'latitude', 'longitude','review_count']].dropna()

coords = np.radians(geo_df[['latitude', 'longitude']].values)

tree = BallTree(coords, metric='haversine')

RADIUS_KM = 0.5
radius_rad = RADIUS_KM / 6371

counts = tree.query_radius(coords, r=radius_rad, count_only=True)

geo_df['restaurants_within_500m'] = counts - 1

L'idée c'est de comprendre est-ce que le restaurant est dans une zone saturée de restaurants ?

Du coup élevé -> zone touristique / commerciale ou faible -> quartier plus résidentiel maybe

## Popularité relative locale

In [41]:
avg_neighbor_reviews = []

for i, neighbors in enumerate(
    tree.query_radius(coords, r=radius_rad)
):
    neighbor_counts = geo_df.iloc[neighbors]['review_count'].values
    avg_neighbor_reviews.append(
        np.mean(neighbor_counts[neighbor_counts > 0])
    )

geo_df['avg_neighbor_review_count'] = avg_neighbor_reviews

geo_df['relative_review_intensity'] = (
    geo_df['review_count'] /
    geo_df['avg_neighbor_review_count']
)


## Centralité

In [43]:
city_center = geo_df[['latitude', 'longitude']].mean().values

geo_df['distance_to_center_km'] = geo_df.apply(
    lambda row: geodesic(
        (row['latitude'], row['longitude']),
        tuple(city_center)
    ).km,
    axis=1
)

Les attrapes touristes ils sotn souvent en centre ville en geéral

## Ajout de ces nouveaux features à notre dataset feature

In [44]:
features_df = features_df.merge(
    geo_df,
    on='business_id',
    how='left'
)

# Visualisation du nouveau dataset feature

In [45]:
features_df.dtypes

business_id                          object
n_reviews                             int64
avg_stars                           float64
std_stars                           float64
first_review_x               datetime64[ns]
last_review_x                datetime64[ns]
activity_span_days_x                  int64
velocity_reviews                    float64
n_star_1                              int64
n_star_2                              int64
n_star_3                              int64
n_star_4                              int64
n_star_5                              int64
share_5_star                        float64
share_1_star                        float64
monthly_review_volatility           float64
first_review_y               datetime64[ns]
last_review_y                datetime64[ns]
activity_span_days_y                  int64
reviews_per_month                   float64
name                                 object
address                              object
city                            